In [11]:
%load_ext autoreload 
%autoreload 2 
import usda.mpe_realworld  as usda_mpe
from usda.mpe_realworld.mpe import navona_v1
import usda.utils as usda_utils
import usda.rl as usda_rl   

import matplotlib.pyplot as plt
import mapclassify
import matplotlib
from IPython.display import HTML
import supersuit as ss

from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3.ppo import MlpPolicy
import os
import rioxarray as rxr
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
piazza_navona_shp_root=r'C:\Users\richie\omen_richiebao\omen_github\USDA_special_study\data\piazza_navona_osm'
area_node_fn=os.path.join(piazza_navona_shp_root,'navona_area_node.tif')
navona_area_node=rxr.open_rasterio(area_node_fn)
plat=np.stack(navona_area_node.data,axis=2)
plat[:,:,0][plat[:,:,0]==-9999]=15   
plat=plat[100:500,100:500:,:]

plat_colors=usda_utils.cmap2hex('Pastel1',16)
print(plat_colors)    
cmap, norm = matplotlib.colors.from_levels_and_colors([i+1 for i in list(plat_colors.keys())],list(plat_colors.values()),extend='max') 

plat_rewards={
    1:300,
    2:-100,
    3:-100,
    4:-100,
    5:-100,
    6:-100,
    7:-100,
    8:-100,
    9:-100,
    10:-100,
    11:10,
    12:-100,
    13:-100,
    14:-100,
    15:300}

{0: '#fbb4ae', 1: '#fbb4ae', 2: '#b3cde3', 3: '#b3cde3', 4: '#ccebc5', 5: '#decbe4', 6: '#decbe4', 7: '#fed9a6', 8: '#fed9a6', 9: '#ffffcc', 10: '#e5d8bd', 11: '#e5d8bd', 12: '#fddaec', 13: '#fddaec', 14: '#f2f2f2', 15: '#f2f2f2'}


In [13]:
env=navona_v1.env(
    render_mode="rgb_array",
    plat=plat,
    plat_colors=plat_colors,
    plat_rewards=plat_rewards,
    agents_num=100,
    nodes_radius=50,
    group_dis=0.01,
    max_cycles=100, 
    # continuous_actions=True,
    )

# navona_v1_PPO_path=r'../models/navona_v1_PPO.zip'
navona_v1_PPO_path=r'C:\Users\richie\omen_richiebao\omen_temp\navona_v1_A2C.zip'
#model = PPO.load(navona_v1_PPO_path)
model = A2C.load(navona_v1_PPO_path)

rewards = {agent: 0 for agent in env.possible_agents}
frames=[]
for i in range(1):
    env.reset(seed=i)
    
    for agent in env.agent_iter():
        frames.append(env.render())
        obs, reward, termination, truncation, info = env.last()
        for agent in env.agents:
            rewards[agent] += env.rewards[agent]
        if termination or truncation:
            break
        else:
            act = model.predict(obs, deterministic=True)[0]

        env.step(act)
        
env.close()
avg_reward = sum(rewards.values()) / len(rewards.values())
print("Rewards: ", rewards)
print(f"Avg reward: {avg_reward}")

Rewards:  {'agent_0': 12391.563888873456, 'agent_1': 16971.56388887344, 'agent_2': -3908.436111126584, 'agent_3': 18301.56388887342, 'agent_4': 5061.563888873428, 'agent_5': -5208.436111126575, 'agent_6': 34421.563888873374, 'agent_7': 7381.56388887345, 'agent_8': 29141.56388887337, 'agent_9': 51831.563888873476, 'agent_10': 15021.563888873447, 'agent_11': 7331.563888873428, 'agent_12': 21131.56388887334, 'agent_13': -8168.436111126562, 'agent_14': 2501.5638888734165, 'agent_15': 12881.563888873456, 'agent_16': 22061.5638888734, 'agent_17': 10301.563888873445, 'agent_18': 1586.5108055396183, 'agent_19': 641.5638888734156, 'agent_20': 16341.563888873447, 'agent_21': 11591.563888873437, 'agent_22': 26406.510805539612, 'agent_23': 50711.56388887346, 'agent_24': 13671.563888873432, 'agent_25': 35461.56388887338, 'agent_26': -5738.436111126572, 'agent_27': 25261.56388887337, 'agent_28': 31731.56388887339, 'agent_29': 27661.563888873367, 'agent_30': 23771.563888873374, 'agent_31': 36561.5638

In [14]:
matplotlib.rcParams['animation.embed_limit'] = 2**128

#anim=usda_rl.plot_animation(frames,interval=1000,figsize=(10,10))
# anim.save(filename="../imgs/3_9_c/navona_v1_A2C.gif")
#HTML(anim.to_jshtml())

In [15]:
anim=usda_rl.plot_animation(frames,interval=100,figsize=(5,5))
anim.save(filename="../imgs/3_9_c/navona_v1_A2C.gif")

MovieWriter ffmpeg unavailable; using Pillow instead.
